In [1]:
import ReportTimerange as rtr
import ReportMailer as rm
import GuestbookDbConnect as gdb

In [2]:
# Install a conda package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install xlsxwriter

# Reporting Period #
* PD - yesterday
* PW - previous week
* PM - previous month
* PQ - previous quarter
* PY - previous calendar year
* YTD year to date
* WTD current week to date
* TODAY today
* P90 - previous 90days

In [3]:
acronyms = [ "PD", "PW", "PM", "PQ", "PY", "YTD", "WTD", "TODAY", "P90"]
friendly = [ "Prior Day", "Prior Week", "Prior Month", "Prior Quarter", "Prior Year", "Year to Date", "Week to Date", "Today", "Previous 90 days"]

def toPeriodFriendly(INTERVAL):
    for n in range(0, len(acronyms)):
      if INTERVAL==acronyms[n]:
         return friendly[n]
    return "Unknown Period"

In [4]:
_INTERVAL = "PQ"

In [5]:
from pandas import ExcelWriter
from pandas import ExcelFile
import xlsxwriter
import datetime

def createSpreadsheetAndMailIt(collections, reportname, recipients, subject, body):
  now = datetime.datetime.now().strftime('%Y-%m-%d_%H%M%S')
  filename = reportname + "-" + now + ".xlsx"
  #writer = ExcelWriter(filename)
  writer = ExcelWriter(filename, engine='xlsxwriter')
  workbook = writer.book  
  print("Writing dataframe to Excel file {0}".format(filename))
  for collection in collections:
    print("Writing {} to spreadsheet".format(collection["name"]))
    collection["dataframe"].to_excel(writer, sheet_name=collection["name"], index=False)
    worksheet = writer.sheets[collection["name"]]
    if 'colwidths' in collection:
      colwidths = collection['colwidths']
      print("colwidths={}".format(colwidths))
      for n in range(0, len(colwidths)):
        col = str(chr(65 + n))
        worksheet.set_column('{}:{}'.format(col,col), colwidths[n])
    else:
      worksheet.set_column('A:A', 30)
  writer.save()
  for recipient in recipients: 
    rm.mailer(recipient, subject, body, filename)
  print("Excel/Email Done!")

In [6]:
import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd

conn = gdb.guestbookDbConnect()

Welcome to Jupyter Notebook.  You are connected to the Opportunity House guestbook database!


In [7]:
trange = rtr.timerange(_INTERVAL)
print(trange)

collections = []
summaries = []

collection = {}
collection["name"] = "Outline"

df = pd.DataFrame({"Report Tab":[ \
              "Summary", \
              "Distinct Clients", \
              "Services", \
              "Housing", \
              "Gender", \
              "Ethnicity", \
              "Veterans", \
              "New Clients"
                                 ], \
                   "Description":[ \
              "summary totals for the report period", \
              "a count of individuals served by sub-period within the period.", \
              "breakdown by service type and sub-period within the period.", \
              "breakdown by housing conditions and sub-period within the period.", \
              "breakdown by gender and sub-period within the period.", \
              "breakdown by ethnicity and sub-period within the period.", \
              "breakdown by veteran status and sub-period within the period.", \
              "a list of clients with first visit occurring in the report period."
                                 ]})

collection["dataframe"] = df 
collection["colwidths"] = [30, 100]
collections.append(collection)



df.head(1000)

[datetime.datetime(2021, 4, 1, 0, 0), datetime.datetime(2021, 6, 30, 23, 59, 59, 999999), 'YYYY-MM/DD WW']


,Report Tab,Description
0,Summary,summary totals for the report period
1,Distinct Clients,a count of individuals served by sub-period wi...
2,Services,breakdown by service type and sub-period withi...
3,Housing,breakdown by housing conditions and sub-period...
4,Gender,breakdown by gender and sub-period within the ...
5,Ethnicity,breakdown by ethnicity and sub-period within t...
6,Veterans,breakdown by veteran status and sub-period wit...
7,New Clients,a list of clients with first visit occurring i...


## Total distinct clients in report period ##

In [8]:
summary = {}
summary["name"] = "Total distinct clients in report period"

query = "SELECT min(timestamp)AS start, max(timestamp) AS end, \
         COUNT(DISTINCT person_id) AS clients FROM guestbook_personsnapshot \
         WHERE timestamp BETWEEN '{}' AND '{}'".format(trange[0], trange[1])

print(query)

data = pd.read_sql(query, conn)

summary["count"] = data.iloc[0,2]
summaries.append(summary)
print(summaries)

data.head(100)

SELECT min(timestamp)AS start, max(timestamp) AS end,          COUNT(DISTINCT person_id) AS clients FROM guestbook_personsnapshot          WHERE timestamp BETWEEN '2021-04-01 00:00:00' AND '2021-06-30 23:59:59.999999'
[{'name': 'Total distinct clients in report period', 'count': 322}]


,start,end,clients
0,2021-04-01 11:40:25.389394+00:00,2021-06-30 14:10:29.567669+00:00,322


# Distinct clients by sub-interval within the report period #
* Statistics focused on the breadth of individuals that are served during the time period.

In [9]:
collection = {}
collection["name"] = "Distinct Clients"

query = "SELECT to_char(timestamp,'{}') AS period,  \
         COUNT(DISTINCT person.idperson) AS clients \
         FROM guestbook_personsnapshot snapshot \
         JOIN guestbook_person person ON person.idperson=snapshot.person_id \
         WHERE timestamp BETWEEN '{}' AND '{}' \
         GROUP BY period".format(trange[2], trange[0], trange[1])
print(query)

data = pd.read_sql(query, conn)
#data.head(1000)

persons = data.pivot_table('clients', index=['period']).fillna(0).astype(int).reset_index('period')

collection["dataframe"] = persons 
collection["colwidths"] = [30, 20]
collections.append(collection)
persons.head(1000)

SELECT to_char(timestamp,'YYYY-MM/DD WW') AS period,           COUNT(DISTINCT person.idperson) AS clients          FROM guestbook_personsnapshot snapshot          JOIN guestbook_person person ON person.idperson=snapshot.person_id          WHERE timestamp BETWEEN '2021-04-01 00:00:00' AND '2021-06-30 23:59:59.999999'          GROUP BY period


,period,clients
0,2021-04/01 13,23
1,2021-04/02 14,32
2,2021-04/05 14,31
3,2021-04/06 14,33
4,2021-04/08 14,24
5,2021-04/09 15,27
6,2021-04/12 15,32
7,2021-04/13 15,30
8,2021-04/14 15,37
9,2021-04/15 15,26


## Total Services delivered in report period ##

In [10]:
query = "SELECT \
         left(service.name, strpos(service.name, '/') - 1) AS servicename, COUNT(*) AS total FROM guestbook_personsnapshot snapshot\
         JOIN guestbook_personservicerequest servicerequest ON servicerequest.connection_id=snapshot.idsnapshot \
         JOIN guestbook_service service ON service.idservice=servicerequest.service_id \
         WHERE timestamp BETWEEN '{}' AND '{}'  AND service.points<=0 \
         GROUP BY servicename \
         ORDER BY total desc".format(trange[0], trange[1])

print(query)

data = pd.read_sql(query, conn)


for n in range(0, data.shape[0]):  
  summary = {}
  summary["name"]  = data.iloc[n,0] + " services delivered."
  summary["count"] = data.iloc[n,1]
  summaries.append(summary)
print(summaries)
data.head(1000)

SELECT          left(service.name, strpos(service.name, '/') - 1) AS servicename, COUNT(*) AS total FROM guestbook_personsnapshot snapshot         JOIN guestbook_personservicerequest servicerequest ON servicerequest.connection_id=snapshot.idsnapshot          JOIN guestbook_service service ON service.idservice=servicerequest.service_id          WHERE timestamp BETWEEN '2021-04-01 00:00:00' AND '2021-06-30 23:59:59.999999'  AND service.points<=0          GROUP BY servicename          ORDER BY total desc
[{'name': 'Total distinct clients in report period', 'count': 322}, {'name': 'Meals services delivered.', 'count': 1802}, {'name': 'Shower services delivered.', 'count': 700}, {'name': 'Clothing services delivered.', 'count': 362}, {'name': 'Nurse services delivered.', 'count': 265}, {'name': 'Laundry services delivered.', 'count': 197}, {'name': 'Computer services delivered.', 'count': 129}, {'name': 'Housing Assistance services delivered.', 'count': 83}, {'name': 'Pastor Consultation se

,servicename,total
0,Meals,1802
1,Shower,700
2,Clothing,362
3,Nurse,265
4,Laundry,197
5,Computer,129
6,Housing Assistance,83
7,Pastor Consultation,39
8,Group Class,7
9,Toiletrie,4


# Services delivered by sub-interval within the report period #

In [11]:
collection = {}
collection["name"] = "Services"

#query = "SELECT to_char(timestamp,'{}') AS period, \
#         service.name AS servicename, COUNT(*) AS total FROM guestbook_personsnapshot snapshot\
#         JOIN guestbook_personservicerequest servicerequest ON servicerequest.connection_id=snapshot.idsnapshot \
#         JOIN guestbook_service service ON service.idservice=servicerequest.service_id \
#         WHERE timestamp BETWEEN '{}' AND '{}' \
#         GROUP BY period, servicename".format(trange[2], trange[0], trange[1])

query = "SELECT to_char(timestamp,'{}') AS period, \
         left(service.name, strpos(service.name, '/') - 1) AS servicename, COUNT(*) AS total FROM guestbook_personsnapshot snapshot\
         JOIN guestbook_personservicerequest servicerequest ON servicerequest.connection_id=snapshot.idsnapshot \
         JOIN guestbook_service service ON service.idservice=servicerequest.service_id \
         WHERE timestamp BETWEEN '{}' AND '{}' AND service.points<=0 \
         GROUP BY period, servicename".format(trange[2], trange[0], trange[1])



print(query)

data = pd.read_sql(query, conn)

services = data.pivot_table('total', index=['period'], columns='servicename').fillna(0).astype(int).reset_index('period')
collection["dataframe"] = services 
collection["colwidths"] = [30, 20, 20, 20, 20, 20, 20, 20]
collections.append(collection)

services.head(1000)

SELECT to_char(timestamp,'YYYY-MM/DD WW') AS period,          left(service.name, strpos(service.name, '/') - 1) AS servicename, COUNT(*) AS total FROM guestbook_personsnapshot snapshot         JOIN guestbook_personservicerequest servicerequest ON servicerequest.connection_id=snapshot.idsnapshot          JOIN guestbook_service service ON service.idservice=servicerequest.service_id          WHERE timestamp BETWEEN '2021-04-01 00:00:00' AND '2021-06-30 23:59:59.999999' AND service.points<=0          GROUP BY period, servicename


servicename,period,Clothing,Computer,Group Class,Housing Assistance,Laundry,Meals,Nurse,Pastor Consultation,Shower,Toiletrie
0,2021-04/01 13,3,0,0,0,6,22,2,0,7,0
1,2021-04/02 14,7,3,0,0,1,25,8,0,10,0
2,2021-04/05 14,5,4,0,0,2,28,0,0,9,0
3,2021-04/06 14,11,3,0,2,4,32,0,2,13,0
4,2021-04/08 14,5,3,0,3,1,21,0,0,6,0
5,2021-04/09 15,5,3,0,0,2,25,6,0,12,0
6,2021-04/12 15,7,5,0,0,5,30,8,1,8,0
7,2021-04/13 15,9,5,0,0,6,29,4,1,11,0
8,2021-04/14 15,0,4,0,0,1,31,5,0,9,0
9,2021-04/15 15,7,1,0,2,1,23,2,1,9,0


# Housing totals within the report period#

In [12]:
query = "SELECT  \
         left(housing.name, strpos(housing.name, '(') - 1)  AS hresponse, COUNT(*) AS total FROM guestbook_personsnapshot snapshot\
         JOIN guestbook_personsurvey survey ON survey.connection_id=snapshot.idsnapshot \
         JOIN guestbook_housingresponse housing ON housing.idhousing=survey.object_id \
         WHERE timestamp BETWEEN '{}' AND '{}' AND prompt_id=8 \
         GROUP BY hresponse".format(trange[0], trange[1])

print(query)

data = pd.read_sql(query, conn)

for n in range(0, data.shape[0]):  
  summary = {}
  summary["name"]  = " Days spent in " + data.iloc[n,0] 
  summary["count"] = data.iloc[n,1]
  summaries.append(summary)
print(summaries)

data.head(1000)

SELECT           left(housing.name, strpos(housing.name, '(') - 1)  AS hresponse, COUNT(*) AS total FROM guestbook_personsnapshot snapshot         JOIN guestbook_personsurvey survey ON survey.connection_id=snapshot.idsnapshot          JOIN guestbook_housingresponse housing ON housing.idhousing=survey.object_id          WHERE timestamp BETWEEN '2021-04-01 00:00:00' AND '2021-06-30 23:59:59.999999' AND prompt_id=8          GROUP BY hresponse
[{'name': 'Total distinct clients in report period', 'count': 322}, {'name': 'Meals services delivered.', 'count': 1802}, {'name': 'Shower services delivered.', 'count': 700}, {'name': 'Clothing services delivered.', 'count': 362}, {'name': 'Nurse services delivered.', 'count': 265}, {'name': 'Laundry services delivered.', 'count': 197}, {'name': 'Computer services delivered.', 'count': 129}, {'name': 'Housing Assistance services delivered.', 'count': 83}, {'name': 'Pastor Consultation services delivered.', 'count': 39}, {'name': 'Group Class service

,hresponse,total
0,Unstable housing,306
1,Jail or priso,5
2,Sheltered,297
3,Stable housing,303
4,Unsheltered,894


# Housing by sub-interval within the report period#
* Answers to the prompt 'Where did you spend last night?'

In [13]:
collection = {}
collection["name"] = "Housing"

#query = "SELECT to_char(timestamp,'{}') AS period, \
#         housing.name AS hresponse, COUNT(*) AS total FROM guestbook_personsnapshot snapshot\
#         JOIN guestbook_personsurvey survey ON survey.connection_id=snapshot.idsnapshot \
#         JOIN guestbook_housingresponse housing ON housing.idhousing=survey.object_id \
#         WHERE timestamp BETWEEN '{}' AND '{}' AND prompt_id=8 \
#         GROUP BY period, hresponse".format(trange[2], trange[0], trange[1])

query = "SELECT to_char(timestamp,'{}') AS period, \
         left(housing.name, strpos(housing.name, '(') - 1) AS hresponse, COUNT(*) AS total FROM guestbook_personsnapshot snapshot\
         JOIN guestbook_personsurvey survey ON survey.connection_id=snapshot.idsnapshot \
         JOIN guestbook_housingresponse housing ON housing.idhousing=survey.object_id \
         WHERE timestamp BETWEEN '{}' AND '{}' AND prompt_id=8 \
         GROUP BY period, hresponse".format(trange[2], trange[0], trange[1])

print(query)

data = pd.read_sql(query, conn)

housing = data.pivot_table('total', index=['period'], columns='hresponse').fillna(0).astype(int).reset_index('period')
#housing = housing.fillna(0)
collection["dataframe"] = housing  
collection["colwidths"] = [30, 30, 30, 30, 30, 30]
collections.append(collection)
housing.head(1000)

SELECT to_char(timestamp,'YYYY-MM/DD WW') AS period,          left(housing.name, strpos(housing.name, '(') - 1) AS hresponse, COUNT(*) AS total FROM guestbook_personsnapshot snapshot         JOIN guestbook_personsurvey survey ON survey.connection_id=snapshot.idsnapshot          JOIN guestbook_housingresponse housing ON housing.idhousing=survey.object_id          WHERE timestamp BETWEEN '2021-04-01 00:00:00' AND '2021-06-30 23:59:59.999999' AND prompt_id=8          GROUP BY period, hresponse


hresponse,period,Jail or priso,Sheltered,Stable housing,Unsheltered,Unstable housing
0,2021-04/01 13,0,4,3,10,3
1,2021-04/02 14,0,5,10,9,6
2,2021-04/05 14,1,6,3,9,6
3,2021-04/06 14,0,9,3,10,11
4,2021-04/08 14,0,9,1,10,4
5,2021-04/09 15,1,2,7,10,6
6,2021-04/13 15,0,5,6,14,5
7,2021-04/14 15,0,11,1,17,6
8,2021-04/15 15,0,4,3,10,9
9,2021-04/16 16,0,5,5,7,7


# Clients (by name) Reporting Unsheltered Housing #
* more than once in the report period

In [14]:
collection = {}
collection["name"] = "Unsheltered_by_Name"

unsheltered = 5
threshold = 1

#query = "SELECT to_char(timestamp,'{}') AS period, \
#         housing.name AS hresponse, COUNT(*) AS total FROM guestbook_personsnapshot snapshot\
#         JOIN guestbook_personsurvey survey ON survey.connection_id=snapshot.idsnapshot \
#         JOIN guestbook_housingresponse housing ON housing.idhousing=survey.object_id \
#         WHERE timestamp BETWEEN '{}' AND '{}' AND prompt_id=8 \
#         GROUP BY period, hresponse".format(trange[2], trange[0], trange[1])

query = "SELECT person.firstname, person.lastname, person.aliasname, \
         COUNT(*) AS unsheltered_nights FROM guestbook_personsnapshot snapshot\
         JOIN guestbook_person person ON person.idperson=snapshot.person_id \
         JOIN guestbook_personsurvey survey ON survey.connection_id=snapshot.idsnapshot \
         JOIN guestbook_housingresponse housing ON housing.idhousing=survey.object_id \
         WHERE timestamp BETWEEN '{}' AND '{}' AND prompt_id=8 AND object_id={} \
         GROUP BY person.idperson ORDER BY COUNT(*) desc".format(trange[0], trange[1], unsheltered)

print(query)

data = pd.read_sql(query, conn)
data.head(100)

#housing = data.pivot_table('total', index=['period'], columns='hresponse').fillna(0).astype(int).reset_index('period')
collection["dataframe"] = data 
collection["colwidths"] = [30, 30, 30, 30, 30, 30]
collections.append(collection)
data.head(1000)

SELECT person.firstname, person.lastname, person.aliasname,          COUNT(*) AS unsheltered_nights FROM guestbook_personsnapshot snapshot         JOIN guestbook_person person ON person.idperson=snapshot.person_id          JOIN guestbook_personsurvey survey ON survey.connection_id=snapshot.idsnapshot          JOIN guestbook_housingresponse housing ON housing.idhousing=survey.object_id          WHERE timestamp BETWEEN '2021-04-01 00:00:00' AND '2021-06-30 23:59:59.999999' AND prompt_id=8 AND object_id=5          GROUP BY person.idperson ORDER BY COUNT(*) desc


,firstname,lastname,aliasname,unsheltered_nights
0,RANDY,HARRINGTON,RANDYH,49
1,STEPHANIE,HELMS,STEPH,46
2,CHARLEY,BLISSIT JR,KAKAROTT,38
3,NICHOLAS,HARRY,NUFERIOUS32,37
4,AARON,AYERS,AARONA,28
5,ROGER,TAYLOR,ROGERTAYLOR1,27
6,RANDALL,MILLER,RANDALLM,24
7,LARRITT,SHEPPARD,LARRITT,24
8,MARGARET,KEE,MISSEY,22
9,DAVID,STIREWALT,HAWK,20


# Gender totals in the report period #

In [15]:
query = "SELECT  \
         gender.name AS response, COUNT(DISTINCT person.idperson) AS total FROM guestbook_personsnapshot snapshot\
         JOIN guestbook_person person ON person.idperson=snapshot.person_id \
         JOIN guestbook_genderresponse gender ON gender.idgender=person.gender_id \
         WHERE timestamp BETWEEN '{}' AND '{}' \
         GROUP BY response".format(trange[0], trange[1])

print(query)
#summaries = []

data = pd.read_sql(query, conn)

for n in range(0, data.shape[0]):  
  summary = {}
  summary["name"]  = "Gender -" + data.iloc[n,0] 
  summary["count"] = data.iloc[n,1]
  summaries.append(summary)
print(summaries)

data.head(100)

SELECT           gender.name AS response, COUNT(DISTINCT person.idperson) AS total FROM guestbook_personsnapshot snapshot         JOIN guestbook_person person ON person.idperson=snapshot.person_id          JOIN guestbook_genderresponse gender ON gender.idgender=person.gender_id          WHERE timestamp BETWEEN '2021-04-01 00:00:00' AND '2021-06-30 23:59:59.999999'          GROUP BY response
[{'name': 'Total distinct clients in report period', 'count': 322}, {'name': 'Meals services delivered.', 'count': 1802}, {'name': 'Shower services delivered.', 'count': 700}, {'name': 'Clothing services delivered.', 'count': 362}, {'name': 'Nurse services delivered.', 'count': 265}, {'name': 'Laundry services delivered.', 'count': 197}, {'name': 'Computer services delivered.', 'count': 129}, {'name': 'Housing Assistance services delivered.', 'count': 83}, {'name': 'Pastor Consultation services delivered.', 'count': 39}, {'name': 'Group Class services delivered.', 'count': 7}, {'name': 'Toiletrie se

,response,total
0,Female,54
1,Male,141
2,Unknown,127


# Gender by sub-interval within the report period #

In [16]:
collection = {}
collection["name"] = "Gender"

query = "SELECT to_char(timestamp,'{}') AS period, \
         gender.name AS response, COUNT(DISTINCT person.idperson) AS total FROM guestbook_personsnapshot snapshot\
         JOIN guestbook_person person ON person.idperson=snapshot.person_id \
         JOIN guestbook_genderresponse gender ON gender.idgender=person.gender_id \
         WHERE timestamp BETWEEN '{}' AND '{}' \
         GROUP BY period, response".format(trange[2], trange[0], trange[1])

print(query)

data = pd.read_sql(query, conn)

gender = data.pivot_table('total', index=['period'], columns='response').fillna(0).astype(int).reset_index('period')
collection["dataframe"] = gender
collection["colwidths"] = [30, 20, 20, 20]
#collection["colwidths"] = []
collections.append(collection)
gender.head(1000)

SELECT to_char(timestamp,'YYYY-MM/DD WW') AS period,          gender.name AS response, COUNT(DISTINCT person.idperson) AS total FROM guestbook_personsnapshot snapshot         JOIN guestbook_person person ON person.idperson=snapshot.person_id          JOIN guestbook_genderresponse gender ON gender.idgender=person.gender_id          WHERE timestamp BETWEEN '2021-04-01 00:00:00' AND '2021-06-30 23:59:59.999999'          GROUP BY period, response


response,period,Female,Male,Unknown
0,2021-04/01 13,5,16,2
1,2021-04/02 14,4,21,7
2,2021-04/05 14,5,23,3
3,2021-04/06 14,9,22,2
4,2021-04/08 14,3,20,1
5,2021-04/09 15,6,17,4
6,2021-04/12 15,6,23,3
7,2021-04/13 15,7,22,1
8,2021-04/14 15,8,24,5
9,2021-04/15 15,7,18,1


# Ethnicity totals within the report period #

In [17]:
query = "SELECT  \
         ethnicity.name AS response, COUNT(DISTINCT person.idperson) AS total FROM guestbook_personsnapshot snapshot\
         JOIN guestbook_person person ON person.idperson=snapshot.person_id \
         JOIN guestbook_ethnicityresponse ethnicity ON ethnicity.idethnicity=person.ethnicity_id \
         WHERE timestamp BETWEEN '{}' AND '{}' \
         GROUP BY response".format(trange[0], trange[1])

print(query)
#summaries = []

data = pd.read_sql(query, conn)
for n in range(0, data.shape[0]):  
  summary = {}
  summary["name"]  = "Ethnicity -" + data.iloc[n,0] 
  summary["count"] = data.iloc[n,1]
  summaries.append(summary)
print(summaries)
data.head(1000)

SELECT           ethnicity.name AS response, COUNT(DISTINCT person.idperson) AS total FROM guestbook_personsnapshot snapshot         JOIN guestbook_person person ON person.idperson=snapshot.person_id          JOIN guestbook_ethnicityresponse ethnicity ON ethnicity.idethnicity=person.ethnicity_id          WHERE timestamp BETWEEN '2021-04-01 00:00:00' AND '2021-06-30 23:59:59.999999'          GROUP BY response
[{'name': 'Total distinct clients in report period', 'count': 322}, {'name': 'Meals services delivered.', 'count': 1802}, {'name': 'Shower services delivered.', 'count': 700}, {'name': 'Clothing services delivered.', 'count': 362}, {'name': 'Nurse services delivered.', 'count': 265}, {'name': 'Laundry services delivered.', 'count': 197}, {'name': 'Computer services delivered.', 'count': 129}, {'name': 'Housing Assistance services delivered.', 'count': 83}, {'name': 'Pastor Consultation services delivered.', 'count': 39}, {'name': 'Group Class services delivered.', 'count': 7}, {'na

,response,total
0,Asian,1
1,Black,104
2,Hispanic,6
3,Native American,3
4,Other,9
5,Unknown,85
6,White,114


# Ethnicity by sub-interval within the report period #

In [18]:
collection = {}
collection["name"] = "Ethnicity"

query = "SELECT to_char(timestamp,'{}') AS period, \
         ethnicity.name AS response, COUNT(DISTINCT person.idperson) AS total FROM guestbook_personsnapshot snapshot\
         JOIN guestbook_person person ON person.idperson=snapshot.person_id \
         JOIN guestbook_ethnicityresponse ethnicity ON ethnicity.idethnicity=person.ethnicity_id \
         WHERE timestamp BETWEEN '{}' AND '{}' \
         GROUP BY period, response".format(trange[2], trange[0], trange[1])

print(query)

data = pd.read_sql(query, conn)

ethnicity = data.pivot_table('total', index=['period'], columns='response').fillna(0).astype(int).reset_index('period')
collection["dataframe"] = ethnicity 
collection["colwidths"] = [30, 20, 20, 20, 20, 20, 20, 20]
collections.append(collection)
ethnicity.head(1000)

SELECT to_char(timestamp,'YYYY-MM/DD WW') AS period,          ethnicity.name AS response, COUNT(DISTINCT person.idperson) AS total FROM guestbook_personsnapshot snapshot         JOIN guestbook_person person ON person.idperson=snapshot.person_id          JOIN guestbook_ethnicityresponse ethnicity ON ethnicity.idethnicity=person.ethnicity_id          WHERE timestamp BETWEEN '2021-04-01 00:00:00' AND '2021-06-30 23:59:59.999999'          GROUP BY period, response


response,period,Asian,Black,Hispanic,Native American,Other,Unknown,White
0,2021-04/01 13,1,9,0,0,0,0,13
1,2021-04/02 14,0,10,0,1,1,5,15
2,2021-04/05 14,0,11,0,0,2,2,16
3,2021-04/06 14,0,10,0,0,1,2,20
4,2021-04/08 14,0,6,0,0,0,0,18
5,2021-04/09 15,0,10,0,0,0,3,14
6,2021-04/12 15,0,12,1,0,2,2,15
7,2021-04/13 15,0,12,1,0,1,0,16
8,2021-04/14 15,0,12,1,0,1,0,23
9,2021-04/15 15,0,12,1,0,1,0,12


# Veteran status totals within the report period#

In [19]:
query = "SELECT  \
         yesno.name AS response, COUNT(DISTINCT person.idperson) AS total FROM guestbook_personsnapshot snapshot\
         JOIN guestbook_person person ON person.idperson=snapshot.person_id \
         JOIN guestbook_yesnoresponse yesno ON yesno.idyesno=person.veteran_id \
         WHERE timestamp BETWEEN '{}' AND '{}' \
         GROUP BY response \
         ORDER BY response desc".format(trange[0], trange[1])

print(query)
#summaries = []

data = pd.read_sql(query, conn)

for n in range(0, data.shape[0]):  
  summary = {}
  if data.iloc[n,0]=="Yes":
    summary["name"] = "Veterans"
  elif data.iloc[n,0]=="No":
    summary["name"] = "Non-veterans"
  else:
    summary["name"] = "Veteran status Unknown"
  summary["count"] = data.iloc[n,1]
  summaries.append(summary)
print(summaries)

data.head(1000)

SELECT           yesno.name AS response, COUNT(DISTINCT person.idperson) AS total FROM guestbook_personsnapshot snapshot         JOIN guestbook_person person ON person.idperson=snapshot.person_id          JOIN guestbook_yesnoresponse yesno ON yesno.idyesno=person.veteran_id          WHERE timestamp BETWEEN '2021-04-01 00:00:00' AND '2021-06-30 23:59:59.999999'          GROUP BY response          ORDER BY response desc
[{'name': 'Total distinct clients in report period', 'count': 322}, {'name': 'Meals services delivered.', 'count': 1802}, {'name': 'Shower services delivered.', 'count': 700}, {'name': 'Clothing services delivered.', 'count': 362}, {'name': 'Nurse services delivered.', 'count': 265}, {'name': 'Laundry services delivered.', 'count': 197}, {'name': 'Computer services delivered.', 'count': 129}, {'name': 'Housing Assistance services delivered.', 'count': 83}, {'name': 'Pastor Consultation services delivered.', 'count': 39}, {'name': 'Group Class services delivered.', 'count'

,response,total
0,Yes,10
1,Unknown,150
2,No,162


# Veteran Status by sub-interval within the report period#

In [20]:
collection = {}
collection["name"] = "Veterans"

query = "SELECT to_char(timestamp,'{}') AS period, \
         yesno.name AS response, COUNT(DISTINCT person.idperson) AS total FROM guestbook_personsnapshot snapshot\
         JOIN guestbook_person person ON person.idperson=snapshot.person_id \
         JOIN guestbook_yesnoresponse yesno ON yesno.idyesno=person.veteran_id \
         WHERE timestamp BETWEEN '{}' AND '{}' \
         GROUP BY period, response".format(trange[2], trange[0], trange[1])

print(query)

data = pd.read_sql(query, conn)
#data.head(1000)



veterans  = data.pivot_table('total', index=['period'], columns='response').fillna(0).astype(int).reset_index('period')
veterans.columns = ['period', 'non-veteran', 'unknown','veteran']
collection["dataframe"] = veterans 
collection["colwidths"] = [30, 30, 30, 30]
collections.append(collection)
veterans.head(1000)


SELECT to_char(timestamp,'YYYY-MM/DD WW') AS period,          yesno.name AS response, COUNT(DISTINCT person.idperson) AS total FROM guestbook_personsnapshot snapshot         JOIN guestbook_person person ON person.idperson=snapshot.person_id          JOIN guestbook_yesnoresponse yesno ON yesno.idyesno=person.veteran_id          WHERE timestamp BETWEEN '2021-04-01 00:00:00' AND '2021-06-30 23:59:59.999999'          GROUP BY period, response


,period,non-veteran,unknown,veteran
0,2021-04/01 13,20,3,0
1,2021-04/02 14,25,7,0
2,2021-04/05 14,26,4,1
3,2021-04/06 14,30,2,1
4,2021-04/08 14,21,1,2
5,2021-04/09 15,20,5,2
6,2021-04/12 15,28,3,1
7,2021-04/13 15,27,1,2
8,2021-04/14 15,29,5,3
9,2021-04/15 15,24,1,1


# New Clients within the report period#
* Clients first appearing in the report interval

In [21]:
collection = {}
collection["name"] = "New Clients"

query = "SELECT firstname, lastname, aliasname, timelinestarttime::date AS startdate FROM guestbook_person \
         WHERE timelinestarttime BETWEEN '{}' AND '{}' \
         ORDER BY timelinestarttime asc".format(trange[0], trange[1])

print(query)

data = pd.read_sql(query, conn)
collection["dataframe"] = data
collection["colwidths"] = [30, 30, 30, 30]
collections.append(collection)

data.head(5000)

SELECT firstname, lastname, aliasname, timelinestarttime::date AS startdate FROM guestbook_person          WHERE timelinestarttime BETWEEN '2021-04-01 00:00:00' AND '2021-06-30 23:59:59.999999'          ORDER BY timelinestarttime asc


,firstname,lastname,aliasname,startdate
0,RONALD,SANFORD,0000RONALD_S,2021-04-01
1,CHAD,EUDY,0000CHAD_EUD,2021-04-02
2,CRAIG,SAUNDERS,0000CRAIG_SA,2021-04-02
3,WILLIAM,SEITZ,0000WILLIAM_,2021-04-06
4,DAVID,SPRINGER,0000DAVID_SP,2021-04-09
5,PENNY,MURRAY,0000PENNY_MU,2021-04-09
6,`JOSE,MARQUEZ,0000`JOSE_MA,2021-04-12
7,JOSE,MARQUEZ,0000JOSE_MAR,2021-04-12
8,MICHAEL,HELMS,0001MICHAEL_,2021-04-12
9,JOSHUA,GREEN,0000JOSHUA_G,2021-04-15


# Summary Counts #

In [22]:
collection = {}
summ = pd.DataFrame.from_dict(summaries)
collection["name"] = "Summary"
collection["dataframe"] = summ
collection["colwidths"] = [30, 60]
collections.insert(1, collection)
#collections.append(collection)
summ.head(1000)

,count,name
0,322,Total distinct clients in report period
1,1802,Meals services delivered.
2,700,Shower services delivered.
3,362,Clothing services delivered.
4,265,Nurse services delivered.
5,197,Laundry services delivered.
6,129,Computer services delivered.
7,83,Housing Assistance services delivered.
8,39,Pastor Consultation services delivered.
9,7,Group Class services delivered.


In [24]:
_REPORTNAME = _INTERVAL
_SUBJECT = "Opportunity House - Reports for {}".format(toPeriodFriendly(_INTERVAL))
_BODY    = "Spreadsheet (attached) with reports for period {} through {}.".format(trange[0], trange[1])
_EMAIL_RECIPIENT = ['cprice9739@carolina.rr.com', 'pastor@opphouse.net']
#_EMAIL_RECIPIENT = ['cprice9739@carolina.rr.com']

createSpreadsheetAndMailIt(collections, _REPORTNAME, _EMAIL_RECIPIENT, _SUBJECT, _BODY)

Writing dataframe to Excel file PQ-2021-07-02_142333.xlsx
Writing Outline to spreadsheet
colwidths=[30, 100]
Writing Summary to spreadsheet
colwidths=[30, 60]
Writing Distinct Clients to spreadsheet
colwidths=[30, 20]
Writing Services to spreadsheet
colwidths=[30, 20, 20, 20, 20, 20, 20, 20]
Writing Housing to spreadsheet
colwidths=[30, 30, 30, 30, 30, 30]
Writing Unsheltered_by_Name to spreadsheet
colwidths=[30, 30, 30, 30, 30, 30]
Writing Gender to spreadsheet
colwidths=[30, 20, 20, 20]
Writing Ethnicity to spreadsheet
colwidths=[30, 20, 20, 20, 20, 20, 20, 20]
Writing Veterans to spreadsheet
colwidths=[30, 30, 30, 30]
Writing New Clients to spreadsheet
colwidths=[30, 30, 30, 30]
Emailer Done!
Emailer Done!
Excel/Email Done!
